# Set up

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=XevOWNYAeYbs8Go2slFyThvFYRhs0OE1i3aaTyj8LHo&tc=NiZsVCuOh2t3xSdIyv_TgAwzTCY9DFaylMIcrx43aSY&cc=jTLK8qRIcQWqGmo2KeWDMwi-U5YLRxtFxMdFqNYrS5k

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk5Wi9m8XWDHTS4p0hebwdOpWsyCiLIr_JN7QQJDwtw7h3tFFIYiPeI

Successfully saved authorization token.


In [ ]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

2.11.0


In [ ]:
import folium
print(folium.__version__)

0.14.0


In [ ]:

import numpy as np
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, UpSampling2D
from keras.callbacks import EarlyStopping
from keras.models import Model
from keras.optimizers import Adam

## Test without the instruction

In [ ]:
# Define the Hansen Global Forest Change dataset
image = ee.Image('UMD/hansen/global_forest_change_2020_v1_8')

# Define the input bands and target band
input_bands = ['treecover2000', 'loss', 'gain']
target_band = 'gain'

# Select the input and target bands
inputs = image.select(input_bands)
target = image.select(target_band).rename('target')

# Define the training and testing dataset
image_collection = ee.ImageCollection.fromImages([inputs, target])
train_dataset = image_collection.filterDate('2001-01-01', '2018-12-31')
test_dataset = image_collection.filterDate('2019-01-01', '2020-12-31')


In [ ]:
train_dataset

In [ ]:
# Define a function to convert image to numpy array
def get_numpy_array(img):
  """
  Convert Earth Engine image to numpy array
  """
  img_array = np.stack([img.select(band).toArray() for band in input_bands], axis=-1)
  return img_array

# Define a function to get a batch of image arrays
def get_batch(batch_size, dataset):
  """
  Get a batch of image arrays and target arrays
  """
  # Get a batch of images
  images = dataset.toList(batch_size)
  input_images = []
  target_images = []
  for i in range(batch_size):
    # Get the image as an Earth Engine object
    ee_image = ee.Image(images.get(i))
    # Convert the image to a numpy array
    input_array = get_numpy_array(ee_image)
    target_array = get_numpy_array(ee_image.select(target_band))
    # Append the arrays to the input and target lists
    input_images.append(input_array)
    target_images.append(target_array)
  # Convert the input and target lists to numpy arrays
  input_images = np.array(input_images)
  target_images = np.array(target_images)
  return input_images, target_images

In [ ]:
# Define the input shape of the model
input_shape = (256, 256, len(input_bands))

# Define the CNN model
inputs = Input(shape=input_shape)
x = Conv2D(32, kernel_size=(3, 3), activation='relu')(inputs)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64, kernel_size=(3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation='sigmoid')(x)
model = Model(inputs=inputs, outputs=outputs)

## Train & Predict with the model

In [ ]:
# Define the loss function and optimizer
loss_function = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Compile the model
model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])
epochs = 1000
steps_per_epoch = 100

# Train the model
for epoch in range(epochs):
  for step in range(steps_per_epoch):
    # Get a batch of training data
    input_images, target_images = get_batch(batch_size, train_dataset)
    # Train the model on the batch of data
    loss, accuracy = model.train_on_batch(input_images, target_images)

    #print("test")
    
    # Print the training progress
    print(f'Epoch {epoch + 1}, Step {step + 1}/{steps_per_epoch}, Loss: {loss}, Accuracy: {accuracy}')


  # Evaluate the model on the test dataset after each epoch
  test_input_images, test_target_images = get_batch(batch_size, test_dataset)


  print(test_target_images) #<class 'numpy.ndarray'> of ee.image

  test_loss, test_accuracy = model.evaluate(test_input_images, test_target_images, verbose=0)
  print(f'Epoch {epoch + 1}, Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')


ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "Collection.filter",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "ImageCollection.fromImages",
          "arguments": {
            "images": {
              "arrayValue": {
                "values": [
                  {
                    "functionInvocationValue": {
                      "functionName": "Image.select",
                      "arguments": {
                        "bandSelectors": {
                          "constantValue": [
                            "treecover2000",
                            "loss",
                            "gain"
                          ]
                        },
                        "input": {
                          "functionInvocationValue": {
                            "functionName": "Image.load",
                            "arguments": {
                              "id": {
                  

ValueError: ignored